## LLM con Open AI

In [ ]:
import pandas as pd
from openai import OpenAI
import os
import getpass

In [ ]:
secret_key = getpass.getpass(prompt='Inserisci la chiave: ')

In [ ]:
os.environ['OPENAI_API_KEY']=secret_key

Inizializziamo le api

In [ ]:
client = OpenAI()

Proviamo a fare una chiamata

In [ ]:
completition = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "Sei un'assistente AI"},
    {"role": "user", "content": "Come puoi aiutarmi con pandas?"}
  ]
)

In [ ]:
print(completition.choices[0].message.content)

## Applicazione pratica con LLM

Acquisiamo il dataframe fatture

In [ ]:
#path = r"C:\Users\ianto\Desktop\Corso python\file"

fatture = pd.read_csv(filepath_or_buffer = "Fatture.csv",
        sep = ";",  #separtore del file
        header = None,
        names = ["NumeroFattura","Tipologia","Importo","Iva","IdCliente","Regione","DataFattura","NumeroFornitore"],
        decimal = ",",
        )

fatture['DataFattura'] =  pd.to_datetime(fatture['DataFattura'],
                                         format="%d/%m/%Y"
                                        )  

fatture['NumeroFornitore'] = fatture['NumeroFornitore'].astype('Int64')

Proviamo a chiedere all'LLM di generare un codice pandas

In [ ]:
risposta = client.chat.completions.create(
  model="gpt-4o",
  messages=[
            {"role": "system",
             "content": "Supporti l'utente scrivendo solo codice Python eseguibile con la libreria pandas per rispondere alla sua richiesta" },
            {"role": "user",
             "content": f"""
            Stai lavorando con un dataframe Pandas in Python.
            Il nome del dataframe è df.
            
            Questo è l'elenco delle colonne di df:
            {str(fatture.columns.values)}

            Considera questa richiesta: qual è la fattura più recente?
            
            Segui queste istruzioni:
            1. Converti la richiesta precedente in codice Python eseguibile usando Pandas.
            2. L'ultima riga di codice dovrebbe essere un'espressione Python che può essere chiamata con la funzione `eval()`.
            3. Il codice dovrebbe rappresentare una soluzione alla query.
            4. Restituisci come risposta soltanto il codice Python, non aggiungere altre considerazioni o preamboli
            5. La risposta non deve iniziare con ```python
            6. Non creare dataframe d'esempio
            7. Non fare nessun commento
            8. Rispondi esclusivamente con codice Python
            9. Non usare eval nella risposta
            10. Non creare esempi di dati, usa direttamente nella risposta il nome del dataframe è df 
             """}],
            temperature=0.2)

In [ ]:
print(risposta.choices[0].message.content)

Creiamo una funzione che richiede in input un dataframe e un'interrogazione in linguaggio naturale

In [ ]:
def generete_pandas_code(df, query):
    user_content = f"""
            Stai lavorando con un dataframe Pandas in Python.
            Il nome del dataframe è df.
            
            Questo è l'elenco delle colonne di df:
            {str(df.columns.values)}

            Considera questa richiesta: {query}?
            
            Segui queste istruzioni:
            1. Converti la richiesta precedente in codice Python eseguibile usando Pandas.
            2. L'ultima riga di codice dovrebbe essere un'espressione Python che può essere chiamata con la funzione `eval()`.
            3. Il codice dovrebbe rappresentare una soluzione alla query.
            4. Restituisci come risposta soltanto il codice Python, non aggiungere altre considerazioni o preamboli
            5. La risposta non deve iniziare con ```python
            6. Non creare dataframe d'esempio
            7. Non fare nessun commento
            8. Rispondi esclusivamente con codice Python
            9. Non usare eval nella risposta
            10. Non creare esempi di dati, usa direttamente nella risposta il nome del dataframe è df 
            """
    domanda = client.chat.completions.create(
          model="gpt-4o",
          messages=[
                    {"role": "system",
                    "content": "Supporti l'utente scrivendo solo codice Python eseguibile con la libreria pandas per rispondere alla sua richiesta" },
                    {"role": "user",
                     "content":user_content }
                   ])

    risposta = domanda.choices[0].message.content
    return risposta

In [ ]:
df_input = fatture

In [ ]:
query_input = input("Chiedi la generazione di un codice Pandas. Scrivi x per uscire")

while query_input!="x":
    risposta = generete_pandas_code(df=df_input, query=query_input)
    print(risposta)
    query_input = input("Chiedi la generazione di un codice Pandas. O scrivi x per uscire")

# LLM in locale

In [ ]:
import ollama

Elenco modelli installati in locale

In [ ]:
ollama.list()

Chiamata in locale, può essere molto lenta

In [ ]:
response =  ollama.chat(model="gemma3:1b",
                         messages=[
                                    {"role": "system", "content": "Sei un'assistente AI"},
                                    {"role": "user", "content": "Qual è la capitale dell'italia"}
                                  ]
                       )

In [ ]:
print(response["message"]["content"])